In [2]:
import sys
sys.path.append('test/')

import numpy as np
from scipy.stats import unitary_group
from base_povm import  Povm
from single_qubit_povm import SingleQubitPOVM
from povm_implementation import ProductPVMSimPOVMImplementation
from qiskit.quantum_info import random_density_matrix, Operator

In [4]:
ops = np.arange(24).reshape((6,2,2))
povm1 = Povm(povm_ops=ops)
print(povm1.dimension)

pauli_X = np.array([[0,1],[1,0]])

basis_0 = np.array([1.,0], dtype=complex)
basis_1 = np.array([0,1.], dtype=complex)
basis_plus = 1.0/np.sqrt(2) * (basis_0 + basis_1)
basis_minus = 1.0/np.sqrt(2) * (basis_0 - basis_1)
basis_plus_i = 1.0/np.sqrt(2) * (basis_0 + 1.j * basis_1)
basis_minus_i = 1.0/np.sqrt(2) * (basis_0 - 1.j * basis_1)

q = [0.5, 0.3, 0.2]

Z0 = np.outer(basis_0, basis_0.conj())
Z1 = np.outer(basis_1, basis_1.conj())
X0 = np.outer(basis_plus, basis_plus.conj())
X1 = np.outer(basis_minus, basis_minus.conj())
Y0 = np.outer(basis_plus_i, basis_plus_i.conj())
Y1 = np.outer(basis_minus_i, basis_minus_i.conj())

sqpovm = SingleQubitPOVM(np.array([q[0]* Z0,q[0]* Z1,q[1]* X0,q[1]* X1,q[2]* Y0,q[2]* Y1]))

sqpovm.povm_pauli_decomp


2


[array([0.25, 0.  , 0.  , 0.25]),
 array([ 0.25,  0.  ,  0.  , -0.25]),
 array([0.15, 0.15, 0.  , 0.  ]),
 array([ 0.15, -0.15,  0.  ,  0.  ]),
 array([0.1, 0. , 0.1, 0. ]),
 array([ 0.1,  0. , -0.1,  0. ])]

In [7]:
sqpovm[0]

Operator([[0.5+0.j, 0. +0.j],
          [0. +0.j, 0. +0.j]],
         input_dims=(2,), output_dims=(2,))


In [8]:
dim = 2 
n_out = 6
n_param = n_out*(2*dim-1) - dim**2

np.random.seed(4)
param = np.random.uniform(low=0, high=1, size=n_param)
povm = Povm.from_param(param, dim)
#povm = sqpovm

np.unique(np.array([np.trace(op.data) for op in povm.povm_operators]))

array([5.01299530e-06+1.11641635e-24j, 3.02104770e-05+8.59363292e-25j,
       4.19594048e-05+4.14111853e-26j, 1.24877387e-02-2.07842297e-21j,
       9.89677783e-01-9.92563858e-21j, 9.97757296e-01-1.98538787e-18j])

In [9]:
rho = random_density_matrix(dims=2, seed = 4)
rho


DensityMatrix([[ 0.46571833-8.48656244e-18j, -0.02617055-4.60556496e-01j],
               [-0.02617055+4.60556496e-01j,  0.53428167+8.48656244e-18j]],
              dims=(2,))


In [10]:
p = povm.get_prob(rho)
p

array([4.68781168e-01, 5.24874638e-01, 6.31079159e-03, 1.17603086e-05,
       1.96265836e-06, 1.96791650e-05])

In [11]:
np.real(np.trace(np.dot(povm.get_ops(),rho.data),axis1=1,axis2=2))

array([4.68781168e-01, 5.24874638e-01, 6.31079159e-03, 1.17603086e-05,
       1.96265836e-06, 1.96791650e-05])

In [12]:
%%timeit
povm = Povm.from_param(param, dim)
p = np.real(np.trace(np.dot(povm.get_ops(),rho.data).T))

84 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [13]:
%%timeit
povm = Povm.from_param(param, dim)
p = np.real(np.trace(np.dot(povm.get_ops(),rho.data),axis1=1,axis2=2))

84 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%%timeit
povm = Povm.from_param(param, dim)
p = povm.get_prob(rho)

94.1 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [4]:
%%timeit

for k in range(povm.n_outcomes) :
    y=povm.povm_operators[k].data

337 ns ± 1.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
%%timeit

for op in povm :
    y=op.data

515 ns ± 1.79 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [4]:
%%timeit

for k in range(povm.n_outcomes) :
    y=povm.povm_operators[k].data

627 ns ± 0.665 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
%%timeit

for op in povm :
    y=op

833 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [42]:
%%timeit

for op in povm :
    y=op

989 ns ± 4.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [41]:
%%timeit

for k in range(povm.n_outcomes) :
    y=povm.povm_operators[k].data

620 ns ± 2.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
(0.3*povm + 0.7*sqpovm).check_validity()

TypeError: unsupported operand type(s) for *: 'float' and 'SingleQubitPOVM'

In [8]:
dim = 2
eival = np.random.uniform(low=-5, high=5, size=dim)
x = unitary_group.rvs(dim)#, random_state=seed_obs[i])
obs = x @ np.diag(eival) @ x.T.conj()

_, V = np.linalg.eigh(obs)

sqpovm = SingleQubitPOVM.from_vectors()
print(sqpovm.check_validity())

summed = np.zeros(4, dtype=complex)
for coef in sqpovm.povm_pauli_decomp:
    summed += coef

summed

True


array([ 1.00000000e+00+0.j, -1.11022302e-16+0.j,  0.00000000e+00+0.j,
        0.00000000e+00+0.j])

In [4]:
ops = np.arange(54).reshape((6,3,3))
povm2 = Povm(povm_ops=ops)
print(povm2.dimension)
print(povm1.dimension)



3
2


In [5]:
povm2.print(n=3)

3


3

In [14]:
from dataclasses import dataclass

@dataclass

class Point:

    lat: float

    long: float


p = Point(lat=1,long=2)
p


Point(lat=1, long=2)

In [ ]:
ops = np.arange(24).reshape((6,2,2))
povm1 = SingleQubitPOVM(povm_ops=ops)
print(povm1.dimension)


In [6]:
ops = np.arange(54).reshape((6,3,3))
povm2 = SingleQubitPOVM(povm_ops=ops)
print(povm2.dimension)
print(povm1.dimension)


ValueError: Dimension of Single Qubit POVM operator space should be 2, not 3.

In [24]:
a = np.arange(36.0).reshape((3,3,2,2))
print(a)

[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]

  [[ 8.  9.]
   [10. 11.]]]


 [[[12. 13.]
   [14. 15.]]

  [[16. 17.]
   [18. 19.]]

  [[20. 21.]
   [22. 23.]]]


 [[[24. 25.]
   [26. 27.]]

  [[28. 29.]
   [30. 31.]]

  [[32. 33.]
   [34. 35.]]]]


In [25]:

b = np.arange(9.0).reshape((3,3))
print(b)

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


In [30]:
(np.multiply(a.T,b.T).T).reshape((3,6,2))

array([[[  0.,   0.],
        [  0.,   0.],
        [  4.,   5.],
        [  6.,   7.],
        [ 16.,  18.],
        [ 20.,  22.]],

       [[ 36.,  39.],
        [ 42.,  45.],
        [ 64.,  68.],
        [ 72.,  76.],
        [100., 105.],
        [110., 115.]],

       [[144., 150.],
        [156., 162.],
        [196., 203.],
        [210., 217.],
        [256., 264.],
        [272., 280.]]])

In [17]:
povm_implementation = ProductPVMSimPOVMImplementation(2, np.array([0,0, 1,0.5, 1,1, 1,1,     0,0, 1,0.5, 0.3,0.2, 2,1]))
povm_implementation._build_qc().draw()

┌──────────────────────┐
q_0: ┤ U(theta[0],phi[0],0) ├
     ├──────────────────────┤
q_1: ┤ U(theta[1],phi[1],0) ├
     └──────────────────────┘

In [19]:
povm = povm_implementation.to_povm()
povm.povm_list[1][:]

[Operator([[0.5+0.j, 0. +0.j],
           [0. +0.j, 0. +0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0. +0.j, -0. +0.j],
           [-0. +0.j,  0.5+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.19253779+0.00000000e+00j, 0.09230753-5.04278350e-02j],
           [0.09230753+5.04278350e-02j, 0.05746221-9.13716138e-19j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0.05746221+9.13716138e-19j, -0.09230753+5.04278350e-02j],
           [-0.09230753-5.04278350e-02j,  0.19253779+0.00000000e+00j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.24441706+0.00000000e+00j, 0.03620368-7.33885021e-03j],
           [0.03620368+7.33885021e-03j, 0.00558294-9.66574794e-20j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0.00558294+9.66574794e-20j, -0.03620368+7.33885021e-03j],
           [-0.03620368-7.33885021e-03j,  0.24441706+0.00000000e+00j]],
          input_dims=(2,), output_dims=(2,))]

In [87]:
n_qubit = 1
parameters = np.array(n_qubit * [0., 0., 0.5 * np.pi, 0., 0.5 * np.pi, 0.5 * np.pi, 2.5, 1.5])
cs_implementation = ProductPVMSimPOVMImplementation(n_qubit=n_qubit, parameters=parameters)
cs_implementation.get_parameter_and_shot(shot=100)

[([array([1.57079633, 1.57079633])], 18),
 ([array([1.57079633, 0.        ])], 35),
 ([array([0., 0.])], 47)]

In [88]:
cs_povm = cs_implementation.to_povm()
cs_povm.povm_list[0][:]

[Operator([[0.5+0.j, 0. +0.j],
           [0. +0.j, 0. +0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0. +0.j, -0. +0.j],
           [-0. +0.j,  0.5+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.15+0.j, 0.15+0.j],
           [0.15+0.j, 0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0.15+0.j, -0.15+0.j],
           [-0.15+0.j,  0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[1.000000e-01+0.00000000e+00j, 6.123234e-18-1.00000000e-01j],
           [6.123234e-18+1.00000000e-01j, 1.000000e-01-1.23299735e-34j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 1.000000e-01+1.23299735e-34j, -6.123234e-18+1.00000000e-01j],
           [-6.123234e-18-1.00000000e-01j,  1.000000e-01+0.00000000e+00j]],
          input_dims=(2,), output_dims=(2,))]

In [28]:
sqpovm[:]

[Operator([[0.5+0.j, 0. +0.j],
           [0. +0.j, 0. +0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0. +0.j, 0. +0.j],
           [0. +0.j, 0.5+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.15+0.j, 0.15+0.j],
           [0.15+0.j, 0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0.15+0.j, -0.15-0.j],
           [-0.15+0.j,  0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.1+0.j , 0. -0.1j],
           [0. +0.1j, 0.1+0.j ]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.1+0.j , 0. +0.1j],
           [0. -0.1j, 0.1+0.j ]],
          input_dims=(2,), output_dims=(2,))]

In [57]:
n_qubit = 1
q = np.random.uniform(0,5,size=3 * n_qubit).reshape((n_qubit, 3))
q /= q.sum(axis=1)[:, np.newaxis]

parameters = np.array(n_qubit * [0., 0., 0.5 * np.pi, 0., 0.5 * np.pi, 0.5 * np.pi, 1, 1])
for i in range(n_qubit):
    parameters[i * 8 + 6] = q[i,0]/q[i,2]
    parameters[i * 8 + 7] = q[i,1]/q[i,2]

cs_implementation = ProductPVMSimPOVMImplementation(n_qubit=n_qubit, parameters=parameters)
cs_povm = cs_implementation.to_povm()
for i in range(n_qubit):
    sqpovm = SingleQubitPOVM(np.array([q[i,0]* Z0,q[i,0]* Z1,q[i,1]* X0,q[i,1]* X1,q[i,2]* Y0,q[i,2]* Y1]))
    for k in range(sqpovm.n_outcomes):
        print(np.allclose(cs_povm.povm_list[i][k].data, sqpovm[k].data))


True
True
True
True
True
True


In [37]:
idx = np.arange(cs_povm.povm_list[0].n_outcomes)
np.random.shuffle(idx)

for k1 in range(cs_povm.povm_list[0].n_outcomes):
    for i, k2 in enumerate(idx) : 
        print(k1,k2)
        print(np.allclose(cs_povm.povm_list[0][k1], sqpovm[k2]))
        if np.allclose(cs_povm.povm_list[0][k1], sqpovm[k2]):
            idx = np.delete(idx, i)
            print(idx)
            break


0 3
False
0 5
False
0 4
False
0 1
False
0 0
True
[3 5 4 1 2]
1 3
False
1 5
False
1 4
False
1 1
True
[3 5 4 2]
2 3
False
2 5
False
2 4
False
2 2
True
[3 5 4]
3 3
True
[5 4]
4 5
False
4 4
True
[5]
5 5
True
[]


In [67]:
q = [0.5, 0.1, 0.2, 0.2]

sqpovm = SingleQubitPOVM(np.array([0.15* X0,   # 0
                                   q[0]* Z0,   # 1
                                   q[0]* Z1,   # 2
                                   0.1 * X0,   # 3
                                   0.3 * X1,   # 4
                                   q[2]* Y0,   # 5
                                   0.1 * Y1,   # 6
                                   0.05* X0,   # 7
                                   0.1 * Y1])) # 8

n_del = 0
to_delete=[]

for k in range(sqpovm.n_outcomes):
    k1 = k - n_del
    print(k,k1,sqpovm.n_outcomes)
    for k2 in range(k1+1,sqpovm.n_outcomes):
        if np.allclose(sqpovm[k1] / np.trace(sqpovm[k1]), sqpovm[k2] / np.trace(sqpovm[k2])):
            sqpovm.povm_operators[k1] = Operator(sqpovm[k1]+sqpovm[k2])
            sqpovm.povm_operators.pop(k2)
            sqpovm.n_outcomes-=1

            to_delete.append(k2+n_del)
            print(k1,k2+n_del)

            n_del+=1

            break
for op in sqpovm:
    print(op)

print(to_delete)

sqpovm.check_validity()

0 0 9
0 3
1 0 8
0 7
2 0 7
3 1 7
4 2 7
5 3 7
6 4 7
7 5 7
5 8
8 5 6
Operator([[0.15+0.j, 0.15+0.j],
          [0.15+0.j, 0.15+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.5+0.j, 0. +0.j],
          [0. +0.j, 0. +0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0. +0.j, 0. +0.j],
          [0. +0.j, 0.5+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[ 0.15+0.j, -0.15-0.j],
          [-0.15+0.j,  0.15+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.1+0.j , 0. -0.1j],
          [0. +0.1j, 0.1+0.j ]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.1+0.j , 0. +0.1j],
          [0. -0.1j, 0.1+0.j ]],
         input_dims=(2,), output_dims=(2,))
[3, 7, 8]


True

In [75]:
q = [0.51, 0.1, 0.2, 0.2]

sqpovm = SingleQubitPOVM(np.array([0.15* X0,   # 0
                                   q[0]* Z0,   # 1
                                   q[0]* Z1,   # 2
                                   0.01* X0,   # 3
                                   0.08* X0,   # 4
                                   0.01* Y1,   # 5
                                   0.29 * X1,   # 6
                                   q[2]* Y0,   # 7
                                   0.09* Y1,   # 8
                                   0.05* X0,   # 9
                                   0.05* Y1,   # 10
                                   0.05* Y1])) # 11
k1=0
n_del = 0
to_delete=[]

weird_list = np.zeros(sqpovm.n_outcomes, dtype = int)

while k1 < sqpovm.n_outcomes:
    print(f' k1: {k1} and #outcomes: {sqpovm.n_outcomes}')
    k2 = k1+1
    while k2 < sqpovm.n_outcomes:
        if np.allclose(sqpovm[k1] / np.trace(sqpovm[k1]), sqpovm[k2] / np.trace(sqpovm[k2])):
            sqpovm.povm_operators[k1] = Operator(sqpovm[k1]+sqpovm[k2])
            sqpovm.povm_operators.pop(k2)
            sqpovm.n_outcomes-=1

            to_delete.append(k2+n_del)
            print(k1+weird_list[k1],k2+n_del)
            weird_list[k2:]+=1
            weird_list = np.delete(weird_list,k2)
            print(weird_list)
            n_del+=1
            k2-=1
        k2+=1
    k1+=1

for op in sqpovm:
    print(op)

print(to_delete)

sqpovm.check_validity()

 k1: 0 and #outcomes: 12
0 3
[0 0 0 1 1 1 1 1 1 1 1]
0 4
[0 0 0 2 2 2 2 2 2 2]
0 9
[0 0 0 2 2 2 2 3 3]
 k1: 1 and #outcomes: 9
 k1: 2 and #outcomes: 9
 k1: 3 and #outcomes: 9
5 9
[0 0 0 2 2 2 4 4]
5 10
[0 0 0 2 2 2 5]
5 11
[0 0 0 2 2 2]
 k1: 4 and #outcomes: 6
 k1: 5 and #outcomes: 6
Operator([[0.145+0.j, 0.145+0.j],
          [0.145+0.j, 0.145+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.51+0.j, 0.  +0.j],
          [0.  +0.j, 0.  +0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.  +0.j, 0.  +0.j],
          [0.  +0.j, 0.51+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.1+0.j , 0. +0.1j],
          [0. -0.1j, 0.1+0.j ]],
         input_dims=(2,), output_dims=(2,))
Operator([[ 0.145+0.j, -0.145-0.j],
          [-0.145+0.j,  0.145+0.j]],
         input_dims=(2,), output_dims=(2,))
Operator([[0.1+0.j , 0. -0.1j],
          [0. +0.1j, 0.1+0.j ]],
         input_dims=(2,), output_dims=(2,))
[3, 4, 9, 9, 10, 11]


True

In [90]:
def merge_povm_effects(povm: Povm) -> Povm:

    k1=0
    n_del = 0

    while k1 < povm.n_outcomes:
        print(f' k1: {k1} and #outcomes: {povm.n_outcomes}')
        k2 = k1+1
        while k2 < povm.n_outcomes:
            if np.allclose(povm[k1] / np.trace(povm[k1]), povm[k2] / np.trace(povm[k2])):
                povm.povm_operators[k1] = Operator(povm[k1]+povm[k2])
                povm.povm_operators.pop(k2)
                povm.n_outcomes-=1
                n_del+=1
                k2-=1
            k2+=1
        k1+=1

    assert povm.check_validity(), 'Error in cleaning the POVM'

    return povm

def sort_povm_effects(povm: Povm) -> Povm:
    sorting_values = np.array([(np.trace(op.data), op.data[0,0]) for op in povm], dtype=[('tr', 'float'), ('m00', 'float')])
    idx_sort = np.argsort(sorting_values, order=('tr','m00'))[::-1]
    povm.povm_operators = [povm[idx] for idx in idx_sort]

    return povm

def clean_povm(povm: Povm) -> Povm:
    povm = merge_povm_effects(povm)
    povm = sort_povm_effects(povm)
    return povm

def compare_povms(povm1: Povm, povm2: Povm, tol:float=1e-5) -> bool:
    del tol
    povm1 = clean_povm(povm1)
    povm2 = clean_povm(povm2)

    idx = np.arange(povm2.n_outcomes)

    for k1 in range(povm1.n_outcomes):
        continue_check = False
        for i, k2 in enumerate(idx) : 
            print(k1,k2)
            print(np.allclose(povm1[k1], povm2[k2]))
            if np.allclose(povm1[k1], povm2[k2]):
                idx = np.delete(idx, i)
                print(idx)
                continue_check = True
                break
        if not continue_check :
            return False

    if len(idx)==0:
        return True
    else:
        return False

In [91]:
compare_povms(sqpovm, cs_povm.povm_list[0])

 k1: 0 and #outcomes: 6
 k1: 1 and #outcomes: 6
 k1: 2 and #outcomes: 6
 k1: 3 and #outcomes: 6
 k1: 4 and #outcomes: 6
 k1: 5 and #outcomes: 6
 k1: 0 and #outcomes: 6
 k1: 1 and #outcomes: 6
 k1: 2 and #outcomes: 6
 k1: 3 and #outcomes: 6
 k1: 4 and #outcomes: 6
 k1: 5 and #outcomes: 6
0 0
False
0 1
False
0 2
False
0 3
False
0 4
False
0 5
False


/var/folders/d7/fx92klcd7fncls676l53b83h0000gn/T/ipykernel_63558/1908604622.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  sorting_values = np.array([(np.trace(op.data), op.data[0,0]) for op in povm], dtype=[('tr', 'float'), ('m00', 'float')])


False

In [78]:
sqpovm[0]

Operator([[0.51+0.j, 0.  +0.j],
          [0.  +0.j, 0.  +0.j]],
         input_dims=(2,), output_dims=(2,))


In [79]:
cs_povm.povm_list[0][0]

Operator([[0.5+0.j, 0. +0.j],
          [0. +0.j, 0. +0.j]],
         input_dims=(2,), output_dims=(2,))


In [33]:
idx_sort = np.argsort(np.array([(np.trace(op.data), op.data[0,0]) for op in sqpovm], dtype=[('tr', 'float'), ('m00', 'float')]), order=('tr','m00'))[::-1]

/var/folders/d7/fx92klcd7fncls676l53b83h0000gn/T/ipykernel_63558/2815594634.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  idx_sort = np.argsort(np.array([(np.trace(op.data), op.data[0,0]) for op in sqpovm], dtype=[('tr', 'float'), ('m00', 'float')]), order=('tr','m00'))[::-1]


In [34]:
[sqpovm[idx] for idx in idx_sort]

[Operator([[0.5+0.j, 0. +0.j],
           [0. +0.j, 0. +0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0. +0.j, 0. +0.j],
           [0. +0.j, 0.5+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[ 0.15+0.j, -0.15-0.j],
           [-0.15+0.j,  0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.15+0.j, 0.15+0.j],
           [0.15+0.j, 0.15+0.j]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.1+0.j , 0. -0.1j],
           [0. +0.1j, 0.1+0.j ]],
          input_dims=(2,), output_dims=(2,)),
 Operator([[0.1+0.j , 0. +0.1j],
           [0. -0.1j, 0.1+0.j ]],
          input_dims=(2,), output_dims=(2,))]

In [30]:
np.array([(np.trace(op.data), op.data[0,0]) for op in sqpovm], dtype=[('tr', 'float'), ('m00', 'float')])

/var/folders/d7/fx92klcd7fncls676l53b83h0000gn/T/ipykernel_63558/2810929492.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  np.array([(np.trace(op.data), op.data[0,0]) for op in sqpovm], dtype=[('tr', 'float'), ('m00', 'float')])


array([(0.3, 0.15), (0.5, 0.5 ), (0.5, 0.  ), (0.2, 0.1 ), (0.3, 0.15),
       (0.2, 0.1 )], dtype=[('tr', '<f8'), ('m00', '<f8')])